# Loading Data via the Wikipedia API

## Overview of the Prepared Dataset

- The planned target variable is the **Click-Through Rate (CTR)** of selected Wikipedia pages.  
- Data will be collected for **5,000–10,000 Wikipedia articles** on similar topics (e.g., science) to ensure accurate predictions.  
- The script will retrieve the following **features** for each article:  
  - topic,  
  - summary,  
  - number and list of categories,  
  - article length (in words),  
  - number of links,  
  - number of links in the first section,
  - number of edits and editors,  
  - clicks in,  
  - clicks out,  
  - number of page views over the last 30 days.  
- Based on these data, **derived features** will be created — for example:  
  - one-hot encoding of selected keywords (based on a bag-of-words approach),  
  - title and/or summary embeddings,  
  - ratio of links to total words.  
- Additionally, **external metadata** related to web traffic will be retrieved (e.g., Google Trends data for article titles).  


In [23]:
%cd C:\Users\piecz\PycharmProjects\WdAN_projekt_wikipedia

[WinError 2] Nie można odnaleźć określonego pliku: 'C:\\Users\\piecz\\PycharmProjects\\WdAN_projekt_wikipedia'
C:\Users\piecz\PycharmProjects\pythonProject2\WdAN_projekt


In [ ]:
!pip install wikipedia-api

In [8]:
import pandas as pd
import numpy as np
import wikipediaapi
import sqlite3
import os
import requests
from IPython.display import display
import time
import random
from requests.exceptions import RequestException
from json import JSONDecodeError
import functions.wiki_db as wiki_db
import importlib
importlib.reload(wiki_db)

<module 'functions.wiki_db' from 'C:\\Users\\piecz\\PycharmProjects\\WdAD_projekt_wikipedia\\functions\\wiki_db.py'>

# Load articles belonging to the given categories

In [15]:
categories_list = ["Cycling activism", "Private transport"]  
df_list = []

for category in categories_list:
    df = wiki_db.get_articles_from_category(category, depth=0)
    df_list.append(df)
    print(f"\nNumber of articles from {category}: {len(df)}")


Number of articles from Cycling activism: 12

Number of articles from Private transport: 9


In [17]:
df_articles = pd.concat(df_list)
display(df_articles.sample(5))
print(f"\nTotal articles collected: {len(df_articles)}")

,title,word_count,num_links_internal,num_categories,categories,num_images,image_titles,num_edits,num_editors,summary,creation_date,mo_page_views
3,Cycling Action Network,459,23,9,[Category:All articles containing potentially ...,4,"[File:Can-logo.png, File:Flag of New Zealand.s...",95,34,Cycling Action Network (CAN) is a national cyc...,2008-03-28T21:12:19Z,114
3,List of automated transit networks suppliers,234,71,7,[Category:All articles with bare URLs for cita...,10,"[File:ATO.svg, File:Flag of Australia (convert...",382,110,This is a list of well-known automated transit...,2014-10-03T19:15:00Z,178
2,Car,5692,500,41,"[Category:19th-century inventions, Category:Al...",10,"[File:07-08 Chevrolet Suburban LT.jpg, File:18...",500,272,"A car, or an automobile, is a motor vehicle wi...",2001-11-01T00:18:55Z,93856
8,Walking,5264,500,17,[Category:All Wikipedia articles in need of up...,10,"[File:0 15e16 17027a85 orig.jpg, File:1987Worl...",500,327,Walking (also known as ambulation) is one of t...,2001-08-19T18:46:26Z,11545
6,I BIKE Dublin,389,24,13,[Category:All Wikipedia articles written in Hi...,4,"[File:Critical Mass Cycle Dublin.jpg, File:IBI...",42,19,I BIKE Dublin is an advocacy group founded in ...,2019-12-04T11:20:05Z,178



Total articles collected: 21


### Annotating info about clicks

In [18]:
path = "C:\\Users\\piecz\\PycharmProjects\\pythonProject2\\WdAN_projekt\\data\\clickstream-enwiki-2024-09.tsv.gz"
articles_of_interest = [x.replace(" ", "_") for x in df_articles.title.tolist()]

clicks_in = pd.Series(0, index=articles_of_interest)
clicks_out = pd.Series(0, index=articles_of_interest)

chunksize = 500_000

for chunk in pd.read_csv(path, sep="\t", header=None, chunksize=chunksize):
    chunk.columns = ["source", "target", "type", "count"]
    
    out_chunk = chunk[chunk["source"].isin(articles_of_interest)]
    out_sum = out_chunk.groupby("source")["count"].sum()
    clicks_out[out_sum.index] += out_sum
    
    in_chunk = chunk[chunk["target"].isin(articles_of_interest)]
    in_sum = in_chunk.groupby("target")["count"].sum()
    clicks_in[in_sum.index] += in_sum

summary = pd.DataFrame({
    "article": articles_of_interest,
    "clicks_in": clicks_in.values,
    "clicks_out": clicks_out.values
})

print(summary)

                                         article  clicks_in  clicks_out
0                               Cycling_advocacy        290          15
1                                      Biketober         20           0
2                        Critical_Mass_(cycling)       3259         265
3                         Cycling_Action_Network         11           0
4                          Dutch_Cycling_Embassy          0           0
5                                     Ghost_bike       3133         378
6                                  I_BIKE_Dublin         50           0
7                                   Kidical_Mass        253          24
8                                 Le_Tour_Entier          0           0
9                                 Ovarian_Psycos         69           0
10                                       Quaxing          0           0
11                   Vision_Zero_(New_York_City)        135           0
12                             Private_transport       2340     

In [19]:
summary.article = summary.article.str.replace("_", " ")

df_articles_summary = df_articles.merge(summary, left_on = "title", right_on = "article")
df_articles_summary = df_articles_summary.drop("article",axis =1)

df_articles_summary["clicks_per_view"] = np.where(
    df_articles_summary["mo_page_views"] != 0,
    df_articles_summary["clicks_out"] / df_articles_summary["mo_page_views"],
    0
)

df_articles_summary.head()

,title,word_count,num_links_internal,num_categories,categories,num_images,image_titles,num_edits,num_editors,summary,creation_date,mo_page_views,clicks_in,clicks_out,clicks_per_view
0,Cycling advocacy,1106,174,13,"[Category:Advocacy groups, Category:Articles w...",5,[File:A Short History of Traffic Engineering.p...,229,139,Cycling advocacy consists of activities that c...,2005-09-16T21:50:36Z,1067,290,15,0.014058
1,Biketober,219,100,5,"[Category:Articles with short description, Cat...",2,"[File:Biketober promotional material.png, File...",6,5,Biketober is a month-long festival that celebr...,2024-09-13T03:11:54Z,75,20,0,0.000000
2,Critical Mass (cycling),1869,152,15,"[Category:Articles containing video clips, Cat...",9,"[File:Commons-logo.svg, File:Critical Mass, Sa...",500,255,Critical Mass is a form of direct action in wh...,2003-01-28T07:28:37Z,5197,3259,265,0.050991
3,Cycling Action Network,459,23,9,[Category:All articles containing potentially ...,4,"[File:Can-logo.png, File:Flag of New Zealand.s...",95,34,Cycling Action Network (CAN) is a national cyc...,2008-03-28T21:12:19Z,114,11,0,0.000000
4,Dutch Cycling Embassy,112,7,4,"[Category:All stub articles, Category:Articles...",1,[File:Flag of the Netherlands.svg],4,2,The Dutch Cycling Embassy is a public-private ...,2025-07-19T18:16:14Z,63,0,0,0.000000


### Adding to the database

In [26]:
db_path = "C:\\Users\\piecz\\PycharmProjects\\WdAD_projekt_wikipedia\\data\\wikipedia_db"
tb_name = "test_19_10_2025"

In [28]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS articles (
    title TEXT,
    word_count INTEGER,
    num_links_internal INTEGER,
    num_categories INTEGER,
    categories TEXT,
    num_images INTEGER,
    image_titles TEXT,
    num_edits INTEGER,
    num_editors INTEGER,
    summary TEXT,
    creation_date TEXT,
    mo_page_views INTEGER,
    clicks_in INTEGER,
    clicks_out INTEGER,
    clicks_per_view REAL
)
""")


conn.commit()
conn.close()
print(f"Database created at: {db_path}")

ProgrammingError: Error binding parameter 5: type 'list' is not supported